In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import scrapbook as sb 
import pandas as pd
import numpy as np 
import seaborn as sns 
import numpy as np
from statistics import mean , median
import matplotlib.pyplot as plt

# Baseline 

In [3]:
books = sb.read_notebooks("./BaseLine_Model_Output")
baseLine_data = []
for nb in books.notebooks: 
    nbList=[nb.scraps['Stats Model MAE'].data,nb.scraps['Catboost MAE'].data]
    baseLine_data.append(nbList)
df = pd.DataFrame(baseLine_data, columns = ["Stats Model","Catboost"])
baseline_data = np.array(baseLine_data)
stats = median(baseline_data[:,0])
catboost = median(baseline_data[:,1])

# GAN Model

In [4]:
book = sb.read_notebooks("./GAN_Output")
gan_data = []
gan_mse = []
for nb in book.notebooks:
    metrics = nb.scraps['GAN_1 Metrics'].data
    for i in range(1000):
        gan_mse.append(metrics[0][i])
    nbList = [nb.scraps['GAN Model MSE'].data,
              nb.scraps['GAN Model MAE'].data,
              nb.scraps['GAN Model Euclidean distance'].data,
              nb.scraps['GAN Model Manhattan Distance'].data]        
    gan_data.append(nbList)
    
df = pd.DataFrame(gan_data, columns = ['MSE','MAE','Euclidean Distance','Manhattan Distance'])
display(df.style)
print("MEDIAN:")
print(df.median(axis = 0))
gan_data = np.array(gan_data)
gan_median = median(gan_data[:,1])
print(gan_median)

,MSE,MAE,Euclidean Distance,Manhattan Distance
0,0.347024,0.424460,2.630746,8.489196
1,0.455544,0.495307,3.017716,9.906132
2,0.712811,0.697429,3.775619,13.948585
3,0.219550,0.384611,2.094750,7.692221
4,0.268144,0.404970,2.315390,8.099402
5,0.310819,0.475616,2.492557,9.512318
6,0.445290,0.423958,2.984065,8.479159
7,0.227665,0.355720,2.133061,7.114405
8,0.379639,0.470970,2.755278,9.419409
9,0.531140,0.471344,3.258814,9.426871


MEDIAN:
MSE                   0.363331
MAE                   0.447715
Euclidean Distance    2.693012
Manhattan Distance    8.954302
dtype: float64
0.4477151076152455


# ABC_GAN Analysis

## ABC Pre-generator - Catboost 


In [5]:
book = sb.read_notebooks("./ABC_GAN_Catboost")
paramVal = [[1,1],[1,0.1],[1,0.01],[0.1,1],[0.1,0.1],[0.1,0.01],[0.01,1],[0.01,0.1],[0.01,0.01]]
abc_mae = [[] for i in range(9)]
abc_mae_skip = [[] for i in range(9)]
abc_mae_mean = [[] for i in range(9)]
abc_mae_skip_mean = [[] for i in range(9)]
abc_weights = [[] for i in range(9)]
prior_model = [[] for i in range(9)]
abc_pre_generator = [[] for i in range(9)]

for nb in book.notebooks:
    metrics1 = np.array(nb.scraps['ABC_GAN_1 Metrics'].data)
    metrics3 = np.array(nb.scraps['ABC_GAN_3 Metrics'].data)
    paramVar = float(nb.papermill_dataframe.iloc[0]['value'])
    paramBias = float(nb.papermill_dataframe.iloc[1]['value'])
    #Divide data according to parameters 
    for i in range(9):
        if paramVar == paramVal[i][0] and paramBias == paramVal[i][1]:
            for j in range(100):
                abc_mae[i].append(metrics1[1,j])
                abc_mae_skip[i].append(metrics3[1,j])
            abc_weights[i].append(nb.scraps['Skip Connection Weight'].data)
            prior_model[i].append(nb.scraps['Prior Model MSE'].data)
            abc_pre_generator[i].append(nb.scraps['ABC Pre-generator MSE'].data)
            abc_mae_skip_mean[i].append(mean(metrics3[1,:]))
            abc_mae_mean[i].append(mean(metrics1[1,:]))

In [6]:

data = [[] for i in range(9)]
for i in range(9):
    for j in range(len(abc_weights[i])):
        data[i].append([paramVal[i][0], paramVal[i][1],prior_model[i][j],
                     abc_pre_generator[i][j],abc_weights[i][j],abc_mae_mean[i][j],abc_mae_skip_mean[i][j]])
    df = pd.DataFrame(data[i], columns = ['Variance','Bias','Prior Model MAE',
                                       'ABC pre-generator MAE','Skip Node weight','ABC GAN MAE','ABC_GAN MAE (skip connection)'])
    display(df.round(5))
    print(df.median(axis=0))
    print("-------------------------------------------------------------")

,Variance,Bias,Prior Model MAE,ABC pre-generator MAE,Skip Node weight,ABC GAN MAE,ABC_GAN MAE (skip connection)
0,1,1,0.55349,1.07389,0.88728,0.39999,0.35076
1,1,1,0.49695,1.47991,0.12838,0.49533,0.51881
2,1,1,0.46301,1.09640,0.11156,0.62987,0.55632
3,1,1,0.37491,1.20632,0.14740,0.31533,0.33148
4,1,1,0.25901,1.39730,0.12584,0.27767,0.33092
5,1,1,0.42910,1.32755,0.59361,0.42387,0.40647
6,1,1,0.25064,1.21604,0.90036,0.31985,0.27739
7,1,1,0.31530,1.47608,0.12112,0.35676,0.37147
8,1,1,0.39262,1.35637,0.32640,0.45790,0.51406
9,1,1,0.29455,1.13716,0.21418,0.37550,0.33416


Variance                         1.000000
Bias                             1.000000
Prior Model MAE                  0.383768
ABC pre-generator MAE            1.271793
Skip Node weight                 0.180790
ABC GAN MAE                      0.387748
ABC_GAN MAE (skip connection)    0.361115
dtype: float64
-------------------------------------------------------------


,Variance,Bias,Prior Model MAE,ABC pre-generator MAE,Skip Node weight,ABC GAN MAE,ABC_GAN MAE (skip connection)
0,1,0.1,0.33982,0.99085,0.73196,0.35385,0.31337
1,1,0.1,0.36078,0.96710,0.42287,0.39859,0.37761
2,1,0.1,0.35341,1.20779,0.08241,0.36900,0.36547
3,1,0.1,0.29915,1.06911,0.08746,0.30682,0.30456
4,1,0.1,0.53808,1.13153,0.15200,0.58616,0.53587
5,1,0.1,0.52993,1.21257,0.52067,0.50569,0.48261
6,1,0.1,0.23190,0.91021,0.72811,0.23282,0.22546
7,1,0.1,0.46407,0.79147,0.06695,0.42181,0.47998
8,1,0.1,0.40931,0.97473,0.08642,0.37943,0.39480
9,1,0.1,0.25540,0.86825,0.07913,0.25027,0.26485


Variance                         1.000000
Bias                             0.100000
Prior Model MAE                  0.357096
ABC pre-generator MAE            0.982791
Skip Node weight                 0.119727
ABC GAN MAE                      0.374218
ABC_GAN MAE (skip connection)    0.371540
dtype: float64
-------------------------------------------------------------


,Variance,Bias,Prior Model MAE,ABC pre-generator MAE,Skip Node weight,ABC GAN MAE,ABC_GAN MAE (skip connection)
0,1,0.01,0.19182,0.94674,0.08454,0.18860,0.21433
1,1,0.01,0.33837,1.08540,0.07246,0.35540,0.36636
2,1,0.01,0.61920,1.28079,0.06696,0.60762,0.60012
3,1,0.01,0.42694,1.13053,0.09642,0.43520,0.42548
4,1,0.01,0.35704,1.01274,0.76609,0.35261,0.36901
5,1,0.01,0.39605,1.17703,0.07694,0.38179,0.41878
6,1,0.01,0.23136,0.89273,0.36493,0.29091,0.24130
7,1,0.01,0.39765,1.11745,0.07427,0.34863,0.40664
8,1,0.01,0.60597,1.29207,0.06909,0.59141,0.65434
9,1,0.01,0.41245,1.05936,0.08070,0.43538,0.41649


...

Variance                         0.010000
Bias                             1.000000
Prior Model MAE                  0.394727
ABC pre-generator MAE            1.023361
Skip Node weight                 0.486114
ABC GAN MAE                      0.438633
ABC_GAN MAE (skip connection)    0.438191
dtype: float64
-------------------------------------------------------------


,Variance,Bias,Prior Model MAE,ABC pre-generator MAE,Skip Node weight,ABC GAN MAE,ABC_GAN MAE (skip connection)
0,0.01,0.1,0.38063,0.37627,0.56189,0.37685,0.38667
1,0.01,0.1,0.29615,0.28527,0.15988,0.30385,0.31401
2,0.01,0.1,0.51166,0.51358,0.23061,0.47840,0.52289
3,0.01,0.1,0.39900,0.37510,0.20257,0.45131,0.46101
4,0.01,0.1,0.39202,0.40724,0.16257,0.40694,0.40043
5,0.01,0.1,0.43905,0.46808,0.75343,0.42816,0.41135
6,0.01,0.1,0.41002,0.40900,0.26527,0.43447,0.45689
7,0.01,0.1,0.34874,0.36578,0.00000,0.35072,0.35708
8,0.01,0.1,0.41720,0.44560,0.49396,0.43446,0.39402
9,0.01,0.1,0.51847,0.55573,0.29779,0.54807,0.55994


Variance                         0.010000
Bias                             0.100000
Prior Model MAE                  0.404513
ABC pre-generator MAE            0.408118
Skip Node weight                 0.247944
ABC GAN MAE                      0.431306
ABC_GAN MAE (skip connection)    0.405891
dtype: float64
-------------------------------------------------------------


,Variance,Bias,Prior Model MAE,ABC pre-generator MAE,Skip Node weight,ABC GAN MAE,ABC_GAN MAE (skip connection)
0,0.01,0.01,0.33028,0.33461,0.03296,0.30637,0.32798
1,0.01,0.01,0.31899,0.32740,0.04498,0.33665,0.31979
2,0.01,0.01,0.32634,0.32176,0.09027,0.34715,0.33142
3,0.01,0.01,0.52266,0.51851,0.05373,0.51330,0.53628
4,0.01,0.01,0.41068,0.41065,0.00000,0.43096,0.41030
5,0.01,0.01,0.36300,0.35787,0.02500,0.34682,0.36348
6,0.01,0.01,0.31797,0.31724,0.00000,0.31611,0.31655
7,0.01,0.01,0.63367,0.63322,0.11209,0.63902,0.63554
8,0.01,0.01,0.43524,0.43527,0.06754,0.45189,0.43657
9,0.01,0.01,0.51606,0.51143,0.06383,0.54559,0.54408


Variance                         0.010000
Bias                             0.010000
Prior Model MAE                  0.386840
ABC pre-generator MAE            0.384259
Skip Node weight                 0.049354
ABC GAN MAE                      0.389054
ABC_GAN MAE (skip connection)    0.386888
dtype: float64
-------------------------------------------------------------


In [7]:
# Display Catboost Summary Tables 
data = np.array(data)
catboostData = []
for i in range(9): 
    catboostData.append([paramVal[i][0], paramVal[i][1],catboost,median(data[i][:,3]),median(data[i][:,5]),median(data[i][:,6]),median(data[i][:,4])])
df = pd.DataFrame(catboostData, columns = ['Variance','Bias','Catboost','Prior Model MAE','mGAN','skipGAN','Skip Node weight'])
display(df.round(5))

,Variance,Bias,Catboost,Prior Model MAE,mGAN,skipGAN,Skip Node weight
0,1.00,1.00,0.35587,1.27179,0.38775,0.36111,0.18079
1,1.00,0.10,0.35587,0.98279,0.37422,0.37154,0.11973
2,1.00,0.01,0.35587,1.10143,0.36859,0.41157,0.07882
3,0.10,1.00,0.35587,0.92371,0.42125,0.46104,0.32040
4,0.10,0.10,0.35587,0.37675,0.37136,0.33398,0.13447
5,0.10,0.01,0.35587,0.46337,0.46298,0.45381,0.06817
6,0.01,1.00,0.35587,1.02336,0.43863,0.43819,0.48611
7,0.01,0.10,0.35587,0.40812,0.43131,0.40589,0.24794
8,0.01,0.01,0.35587,0.38426,0.38905,0.38689,0.04935


## ABC Pre-generator - Stats 


In [8]:
book = sb.read_notebooks("./ABC_GAN_Stats")
paramVal = [[1,1],[0.1,1],[0.01,1],[1,0.1],[0.1,0.1],[0.01,0.1],[1,0.01],[0.1,0.01],[0.01,0.01]]
abc_mae = [[] for i in range(9)]
abc_mae_skip = [[] for i in range(9)]
abc_mae_mean = [[] for i in range(9)]
abc_mae_skip_mean = [[] for i in range(9)]
abc_weights = [[] for i in range(9)]
prior_model = [[] for i in range(9)]
abc_pre_generator = [[] for i in range(9)]

for nb in book.notebooks:
    metrics1 = np.array(nb.scraps['ABC_GAN_1 Metrics'].data)
    metrics3 = np.array(nb.scraps['ABC_GAN_3 Metrics'].data)
    paramVar = float(nb.papermill_dataframe.iloc[0]['value'])
    paramBias = float(nb.papermill_dataframe.iloc[1]['value'])
    #Divide data according to parameters 
    for i in range(9):
        if paramVar == paramVal[i][0] and paramBias == paramVal[i][1]:
            for j in range(100):
                abc_mae[i].append(metrics1[1,j])
                abc_mae_skip[i].append(metrics3[1,j])
            abc_weights[i].append(nb.scraps['Skip Connection Weight'].data)
            prior_model[i].append(nb.scraps['Prior Model MSE'].data)
            abc_pre_generator[i].append(nb.scraps['ABC Pre-generator MSE'].data)
            abc_mae_skip_mean[i].append(mean(metrics3[1,:]))
            abc_mae_mean[i].append(mean(metrics1[1,:]))

In [9]:
data = [[] for i in range(9)]
for i in range(9):
    for j in range(len(abc_weights[i])):
        data[i].append([paramVal[i][0], paramVal[i][1],prior_model[i][j],
                     abc_pre_generator[i][j],abc_weights[i][j],abc_mae_mean[i][j],abc_mae_skip_mean[i][j]])
    
    df = pd.DataFrame(data[i], columns = ['Variance','Bias','Prior Model MAE',
                                       'ABC pre-generator MAE','Skip Node weight','ABC GAN MAE','ABC_GAN MAE (skip connection)'])
    display(df.round(5))
    print(df.median(axis=0))
    print("-------------------------------------------------------------")

,Variance,Bias,Prior Model MAE,ABC pre-generator MAE,Skip Node weight,ABC GAN MAE,ABC_GAN MAE (skip connection)
0,1,1,0.45321,1.37373,0.98985,0.46898,0.40212
1,1,1,0.46372,1.50672,0.97564,0.40372,0.29641
2,1,1,0.58514,1.16563,0.99841,0.44245,0.42443
3,1,1,0.52032,1.42406,1.00000,0.68059,0.77544
4,1,1,0.66686,1.11099,0.99498,0.51811,0.51083
5,1,1,0.59179,1.56344,0.99157,0.97788,0.71474
6,1,1,0.49935,1.69111,1.00000,0.46297,0.32696
7,1,1,0.44243,1.40947,0.98950,0.34726,0.28979
8,1,1,0.49656,1.25013,1.00000,0.61839,0.49452
9,1,1,0.55986,1.11980,0.99020,0.51472,0.60578


Variance                         1.000000
Bias                             1.000000
Prior Model MAE                  0.509832
ABC pre-generator MAE            1.391597
Skip Node weight                 0.993273
ABC GAN MAE                      0.491848
ABC_GAN MAE (skip connection)    0.459476
dtype: float64
-------------------------------------------------------------


,Variance,Bias,Prior Model MAE,ABC pre-generator MAE,Skip Node weight,ABC GAN MAE,ABC_GAN MAE (skip connection)
0,0.1,1,0.56628,1.09693,0.98871,0.53355,0.46986
1,0.1,1,0.64543,1.14204,0.99631,0.64390,0.86049
2,0.1,1,0.46983,0.87978,1.00000,0.47337,0.36084
3,0.1,1,0.43445,0.87695,0.99956,0.43586,0.36840
4,0.1,1,0.66464,1.07654,0.99542,0.36400,0.42828
5,0.1,1,0.47202,0.91590,0.99953,0.39912,0.46462
6,0.1,1,0.43826,1.11128,0.99706,0.33126,0.36497
7,0.1,1,0.38618,0.73310,0.99848,0.42586,0.34843
8,0.1,1,0.58504,0.92217,0.99854,0.53167,0.69129
9,0.1,1,0.49714,0.93450,1.00000,0.48852,0.44818


Variance                         0.100000
Bias                             1.000000
Prior Model MAE                  0.484579
ABC pre-generator MAE            0.928335
Skip Node weight                 0.998511
ABC GAN MAE                      0.454615
ABC_GAN MAE (skip connection)    0.438231
dtype: float64
-------------------------------------------------------------


,Variance,Bias,Prior Model MAE,ABC pre-generator MAE,Skip Node weight,ABC GAN MAE,ABC_GAN MAE (skip connection)
0,0.01,1,0.52735,0.88408,0.99728,0.53225,0.53593
1,0.01,1,0.49734,0.82135,0.99989,0.44240,0.54811
2,0.01,1,0.71581,0.84162,0.99677,0.68773,0.68961
3,0.01,1,0.60416,1.24250,0.99780,0.58669,0.61441
4,0.01,1,0.35647,0.87529,1.00000,0.47759,0.39850
5,0.01,1,0.53557,0.83153,1.00000,0.37981,0.36269
6,0.01,1,0.42949,0.75890,0.99731,0.35582,0.39652
7,0.01,1,0.66363,0.88791,0.97753,0.52959,0.63537
8,0.01,1,0.48557,1.00044,0.97891,0.70407,0.58905
9,0.01,1,0.51627,0.94253,1.00000,0.64813,0.54281


...

Variance                         1.000000
Bias                             0.010000
Prior Model MAE                  0.554083
ABC pre-generator MAE            1.053550
Skip Node weight                 0.343423
ABC GAN MAE                      0.512035
ABC_GAN MAE (skip connection)    0.466998
dtype: float64
-------------------------------------------------------------


,Variance,Bias,Prior Model MAE,ABC pre-generator MAE,Skip Node weight,ABC GAN MAE,ABC_GAN MAE (skip connection)
0,0.1,0.01,0.52713,0.53239,0.24743,0.49321,0.52259
1,0.1,0.01,0.60158,0.58374,0.15920,0.48550,0.52312
2,0.1,0.01,0.63817,0.62046,0.20914,0.55401,0.54453
3,0.1,0.01,0.56911,0.56816,0.18413,0.38670,0.46995
4,0.1,0.01,0.64806,0.61719,0.16301,0.36521,0.36476
5,0.1,0.01,0.61426,0.61155,0.21559,0.67290,0.44144
6,0.1,0.01,0.52819,0.53986,0.21552,0.65886,0.39668
7,0.1,0.01,0.43284,0.40754,0.32789,0.27239,0.27265
8,0.1,0.01,0.47584,0.48683,0.21928,0.51177,0.46258
9,0.1,0.01,0.41747,0.39251,0.20267,0.43027,0.39745


Variance                         0.100000
Bias                             0.010000
Prior Model MAE                  0.548651
ABC pre-generator MAE            0.554013
Skip Node weight                 0.212334
ABC GAN MAE                      0.489353
ABC_GAN MAE (skip connection)    0.452011
dtype: float64
-------------------------------------------------------------


,Variance,Bias,Prior Model MAE,ABC pre-generator MAE,Skip Node weight,ABC GAN MAE,ABC_GAN MAE (skip connection)
0,0.01,0.01,0.31832,0.32375,0.19151,0.30129,0.27470
1,0.01,0.01,0.54027,0.53400,0.17197,0.43375,0.50622
2,0.01,0.01,0.43639,0.43214,0.17381,0.46646,0.39244
3,0.01,0.01,0.49849,0.49781,0.19117,0.36862,0.46545
4,0.01,0.01,0.61684,0.61271,0.13287,0.87588,0.57286
5,0.01,0.01,0.58378,0.57790,0.20367,0.53713,0.35204
6,0.01,0.01,0.48061,0.48210,0.18303,0.46122,0.39844
7,0.01,0.01,0.54218,0.53840,0.17850,0.56682,0.58491
8,0.01,0.01,0.58031,0.57566,0.21411,0.52019,0.54366
9,0.01,0.01,0.51799,0.51483,0.21570,0.42981,108.15121


Variance                         0.010000
Bias                             0.010000
Prior Model MAE                  0.529130
ABC pre-generator MAE            0.524417
Skip Node weight                 0.187100
ABC GAN MAE                      0.463838
ABC_GAN MAE (skip connection)    0.485838
dtype: float64
-------------------------------------------------------------


In [10]:
# Display Stats Summary Tables 
data = np.array(data)
catboostData = []
for i in range(9): 
    catboostData.append([gan_median,stats,paramVal[i][0], paramVal[i][1],median(data[i][:,3]),median(data[i][:,5]),median(data[i][:,6]),median(data[i][:,4])])
df = pd.DataFrame(catboostData, columns = ['GAN','Stats','Variance','Bias','Prior Model','mGAN','skipGAN','Weight'])
display(df.round(5))

,GAN,Stats,Variance,Bias,Prior Model,mGAN,skipGAN,Weight
0,0.44772,0.48174,1.00,1.00,1.39160,0.49185,0.45948,0.99327
1,0.44772,0.48174,0.10,1.00,0.92834,0.45461,0.43823,0.99851
2,0.44772,0.48174,0.01,1.00,0.87969,0.53092,0.54546,0.99755
3,0.44772,0.48174,1.00,0.10,1.09603,0.50607,0.42450,0.66578
4,0.44772,0.48174,0.10,0.10,0.54435,0.55939,0.49485,0.48871
5,0.44772,0.48174,0.01,0.10,0.53001,0.50765,0.44932,0.27469
6,0.44772,0.48174,1.00,0.01,1.05355,0.51204,0.46700,0.34342
7,0.44772,0.48174,0.10,0.01,0.55401,0.48935,0.45201,0.21233
8,0.44772,0.48174,0.01,0.01,0.52442,0.46384,0.48584,0.18710
